# Hugging Face

## 0. Environment Settings 环境配置

- 在conda环境中安装基础python依赖：
    ```bash
    conda create -n transformer
    conda activate transformer
    pip install transformer datasets evaluate peft accelerate gradio optimum sentencepiece jupyterlab scikit-learn pandas matplotlib tensorboard nltk rouge
    ```

- 网络问题更改本地电脑hosts：*本条操作待验证*
  - hosts储存路径为: *C:\Windows\System32\drivers\etc\hosts*
  - 添加如下内容：

## 1. Pipeline 工具

### 1.1 Pipeline 概念


[Pipeline Official Doc](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.pipeline)

- pipeline是一个封装好的工具，可以通过直接的声明进行调用，完成一些预先设定好的NLP任务

- pipeline的大致syntax为（较常用的一些opt参数）：`pipeline(task="", model="", tokenizer="", device=)`

![20230716105734](https://michael-1313341240.cos.ap-shanghai.myqcloud.com/20230716105734.png)


### 1.2 Pipeline支持的Task


- task主要可以包括的任务有：
  - 文本分类（包含情感分析）: `task = "text-classification"` `task = "zero-shot-classification"`
  - Token 分类（包含命名实体识别 NER）: `task = "token-classification"`
  - 人机对话: `task = "conversational"`
  - 文档问答: `task = "document-question-answering"`
  - 问题回答: `task = "question-answering"`
  - 表格回答: `task = "table-question-answering"`
  - 填空: `task = "fill-mask"`
  - 生成摘要: `task = "summarization"`
  - Text-to-text 文本生成: `task = "text2text-generation"`
  - 机器翻译: `task = "translation"`
  - 跨模态任务: `task = "image-to-text"` `task = "visual-question-answering"`


*可以通过下列命令查看 **pipeline 支持的所有任务***

In [5]:
from transformers.pipelines import SUPPORTED_TASKS
print(*SUPPORTED_TASKS.items(),sep="\n")

('audio-classification', {'impl': <class 'transformers.pipelines.audio_classification.AudioClassificationPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForAudioClassification'>,), 'default': {'model': {'pt': ('superb/wav2vec2-base-superb-ks', '372e048')}}, 'type': 'audio'})
('automatic-speech-recognition', {'impl': <class 'transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForCTC'>, <class 'transformers.models.auto.modeling_auto.AutoModelForSpeechSeq2Seq'>), 'default': {'model': {'pt': ('facebook/wav2vec2-base-960h', '55bb623')}}, 'type': 'multimodal'})
('feature-extraction', {'impl': <class 'transformers.pipelines.feature_extraction.FeatureExtractionPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModel'>,), 'default': {'model': {'pt': ('distilbert-base-cased', '935ac13'), 'tf': ('distilbert-base-cased', '9

### 1.3 Pipeline的创建与使用（例）

- 在HuggingFace的[Model](https://huggingface.co/models)分区可以寻找想要用的模型
- 点击进入模型详细页面后，可以看到具体描述和demo交互

![20230716120431](https://michael-1313341240.cos.ap-shanghai.myqcloud.com/20230716120431.png)

In [6]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline
# model是指定这个task要使用的nlp模型
model = AutoModelForSequenceClassification.from_pretrained('uer/roberta-base-finetuned-chinanews-chinese')
# tokenizer是指定task的分词器，其具体含义会在后面进行学习
tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-chinanews-chinese')
# 通过pipeline(...)就会生成这样一个封装好的工具，这里命名为pipe
## 此处的task为sentiment analysis，即情感分析
pipe = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
# 调用pipe，配合相应的输入，就可以得到nlp任务的相应输出
pipe("暴雪被微软收购了！")

KeyboardInterrupt: 

**使用GPU加速计算**

通过`pipeline(, ... , device = 0)`显式加载第0张GPU进行加速计算

In [ ]:
import torch
import time
times = []

# GPU inference
pipe_gpu = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer,device=0)
for i in range(1000):
    torch.cuda.synchronize()
    start = time.time()
    pipe_gpu("暴雪被微软收购了！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(f"GPU:{sum(times) / 1000}")

# CPU inference
for i in range(1000):
    torch.cuda.synchronize()
    start = time.time()
    pipe("暴雪被微软收购了！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(f"CPU:{sum(times) / 1000}")

GPU:0.005329703807830811
CPU:0.01029831838607788


### 1.4 Pipeline的背后实现

**1. 初始化Tokenizer**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained\
    ("uer/roberta-base-finetuned-dianping-chinese")

**2. 初始化Model**

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained\
    ("uer/roberta-base-finetuned-dianping-chinese")

In [ ]:
# 查看模型结构
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

**3. 数据输入&通过`tokenizer`预处理**

In [ ]:
input_txt = "我很喜欢你！"
input = tokenizer(input_txt, return_tensors="pt") # pt即pytorch

In [ ]:
# 查看分词后结果
print(input)

{'input_ids': tensor([[ 101, 2769, 2523, 1599, 3614,  872, 8013,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


**4. 模型预测**

In [ ]:
res = model(**input) # 这一步得到的结果是一个未标准化的logits向量
print(f"res:\n{res}") # 输出res，这里列出了一系列中间操作的说明；数值上重点关注logits值
logits = res.logits # 从res中提取logits向量
print(f"logits:\n{logits}") # 输出logits向量

res:
SequenceClassifierOutput(loss=None, logits=tensor([[-1.4593,  1.4171]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
logits:
tensor([[-1.4593,  1.4171]], grad_fn=<AddmmBackward0>)


**5. 输出结果**

In [ ]:
logits_softmax = torch.softmax(logits,dim=-1) # 对logits向量进行softmax归一化操作
print(f"logits_softmax:\t{logits_softmax}") # 输出归一化后的logits向量

pred = torch.argmax(logits_softmax).item() # 从归一化后的logits向量中提取最大值的索引 
print(f"pred:\t{pred}") # 输出预测结果，这里的pred是一个数字索引，需要转换为索引对应的标签

result = model.config.id2label.get(pred) # 通过id2label字典，将索引转换为标签
print(f"result:\t{result}") # 输出标签

logits_softmax:	tensor([[0.0533, 0.9467]], grad_fn=<SoftmaxBackward0>)
pred:	1
result:	positive (stars 4 and 5)


## 2. Tokenizer 分词器

### 2.1 Tokenizer 概念

**基本NLP数据预处理流程**

1. 分词：将文本数据分词成字、字词等
2. 构建词典：构建词典，filter一些过低、过高的单词等
3. 数据转换：将词典映射成数字序列
4. 数据填充与截断
   

### 2.2 Tokenizer的使用

In [ ]:
from transformers import AutoTokenizer
demo = "那些痛的记忆，落在春的泥土里"

#### Step1. 加载Tokenizer

In [ ]:
# 从huggingface的模型库中加载分词器
tokenizer = AutoTokenizer.from_pretrained\
    ("uer/roberta-base-finetuned-dianping-chinese")
print(tokenizer)

BertTokenizerFast(name_or_path='uer/roberta-base-finetuned-dianping-chinese', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


也可以将tokenizer保存到本地再从本地调用

In [ ]:
tokenizer.save_pretrained("./roberta_tokenizer")
tokenizer = AutoTokenizer.from_pretrained("./roberta_tokenizer")
print(tokenizer)

在调用非官方上传的远程仓库中的tokenizer时，要加入`trust_remote_code=True`参数，例如：

In [ ]:
from transformers import AutoTokenizer
tokenizer_chatglm = AutoTokenizer.from_pretrained\
    ("THUDM/chatglm-6b",trust_remote_code = True)
print(tokenizer_chatglm)

ChatGLMTokenizer(name_or_path='THUDM/chatglm-6b', vocab_size=130344, model_max_length=2048, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<sop>', 'eos_token': '<eop>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


#### Step2. 利用`tokenizer.tokenize()`进行分词

In [ ]:
tokens = tokenizer.tokenize(demo)
print(tokens)

['那', '些', '痛', '的', '记', '忆', '，', '落', '在', '春', '的', '泥', '土', '里']


*对于tokenizer词典，可以通过以下方法进行查看：*

`tokenizer.vocab()`：查看词典

`tokenizer.vocab_size`：查看词典大小

#### Step3\*. 利用`tokenizer.convert_to_ids()`进行索引转换

目的：token -> id

In [ ]:
# 将词序列转换为id序列
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027]


*相反地，也可以通过`tokenizer.convert_ids_to_tokens(ids)`进行逆变换*

In [ ]:
# 将id序列转换为token序列
tokens = tokenizer.convert_ids_to_tokens(ids)
print(tokens)

# 将token序列转换为string
str_sen = tokenizer.convert_tokens_to_string(tokens)
print(str_sen)

['那', '些', '痛', '的', '记', '忆', '，', '落', '在', '春', '的', '泥', '土', '里']
那 些 痛 的 记 忆 ， 落 在 春 的 泥 土 里


#### Step3. 利用`tokenizer.encode()`快捷转换

**作用：token -> id**

In [ ]:
# 将字符串转换为id序列，又称之为编码
ids = tokenizer.encode(demo, add_special_tokens=True)
print(ids)

[101, 6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027, 102]


注：
1. 若和上面的`tokenizer.convert_to_ids()`同时使用，会发现`tokenizer.encode()`的结果会在前后加上`[CLS]`和`[SEP]`，也就是输出中的[101],[102]. 其本身的目的是为了方便进行模型输入的构建。
2. 若不希望加入`[CLS]`和`[SEP]`，可以通过`add_special_tokens=False`进行设置。

相对应，可以通过`tokenizer.decode()`进行逆变换

**作用：id -> token**

In [ ]:
# 将id序列转换为字符串，又称之为解码
str_sen = tokenizer.decode(ids, skip_special_tokens=False)
print(str_sen)

[CLS] 那 些 痛 的 记 忆 ， 落 在 春 的 泥 土 里 [SEP]


#### Step4. 填充与截断

说明：在进行分词的过程中，会有些句子过长或过短，因此需要进行填充与截断以得到长度适中的数据。

**填充**

利用`tokenizer.encode(..., padding = "max_length", max_length = )` 进行填充

In [ ]:
# 填充
ids = tokenizer.encode(demo, padding="max_length", max_length=20)
print(ids)
print(tokenizer.decode(ids, skip_special_tokens=False))

[101, 6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027, 102, 0, 0, 0, 0]
[CLS] 那 些 痛 的 记 忆 ， 落 在 春 的 泥 土 里 [SEP] [PAD] [PAD] [PAD] [PAD]


*注：观察上面的输出可以发现最后几个位置为0，如果decode回去会发现这里是`[PAD]`，也就是填充的位置。*

**截断**

利用`tokenizer.encode(..., truncation = True, max_length = )` 进行截断

In [ ]:
# 截断
ids = tokenizer.encode(demo, max_length=5, truncation=True)
ids

[101, 6929, 763, 4578, 102]

**Attention Mask**

对于某些模型来说，单纯的填充是不够的，需要另外建立一个attention_mask的bool数组来指示那部分是真实输出，那部分是填充内容。

这一功能在调用`tokenizer.encode_plus`时，会自动完成。

In [ ]:
inputs = tokenizer.encode_plus(demo, padding="max_length", max_length=15)
print(inputs)

{'input_ids': [101, 6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


等价地，上述全部流程最后被完全封装进了`tokenizer`中，因此在实际操作中，可以直接调用`tokenizer`进行数据预处理。

In [ ]:
inputs = tokenizer(demo, padding="max_length", max_length=15)
print(inputs)

{'input_ids': [101, 6929, 763, 4578, 4638, 6381, 2554, 8024, 5862, 1762, 3217, 4638, 3799, 1759, 7027, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


#### Step 5. Batch数据的处理

`tokenizer()`支持对batch数据的处理，只需要将数据以list的形式传入即可。

In [ ]:
demo2 = ["那些痛的记忆","落在春的泥土里","滋养了大地，开出下一个花季"]
res= tokenizer(demo2)
print(res)

{'input_ids': [[101, 6929, 763, 4578, 4638, 6381, 2554, 102], [101, 5862, 1762, 3217, 4638, 3799, 1759, 7027, 102], [101, 3996, 1075, 749, 1920, 1765, 8024, 2458, 1139, 678, 671, 702, 5709, 2108, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


### 2.3 Fast&Slow Tokenizer

在最开始调用tokenizer的时候，如果输出引入的分词器，可以看到有如下输出结果：

In [ ]:
print(tokenizer)

BertTokenizerFast(name_or_path='uer/roberta-base-finetuned-dianping-chinese', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


注意到这里我们使用的是`BertTokenizerFast`；因此便引入了Fast 与 Slow Tokenizer的比较。

简而言之，Fast Tokenizer是基于Rust语言的实现；而Slow Tokenizer是基于python实现的。

在引入tokenizer的时候，可以通过`AutoTokenizer.from_pretrained(...,use_fast = False)`来指定使用Slow Tokenizer。

## 3. Model 模型调用

### 3.1 模型的加载

**直接调用python命令**

In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
# model = AutoModel.from_pretrained("hfl/rbt3", force_download=True)
model = AutoModel.from_pretrained("hfl/rbt3")

**加载本地下载好的Model**

In [ ]:
model = AutoModel.from_pretrained("rbt3")

**通过git clone模型仓库**

In [ ]:
!git lfs clone "https://huggingface.co/hfl/rbt3" --include="*.bin"

*通过`lfs ... --include`指定需要下载的文档格式*

### 3.2 模型的config参数

In [ ]:
config = AutoConfig.from_pretrained("hfl/rbt3")
config

### 3.3 模型调用

In [ ]:
sen = "充满鲜花的世界到底在哪里"
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")
inputs = tokenizer(sen, return_tensors="pt")
inputs

In [ ]:
model = AutoModel.from_pretrained("hfl/rbt3", output_attentions=True)

In [ ]:
output = model(**inputs)
output

## 5. Dataset

### 5.1 加载在线数据集

In [ ]:
from datasets import *

#### Basic Loading

[HuggingFace_datasets](https://huggingface.co/datasets)

注意，在下载数据集时易出现网络问题，需要科学上网。

这里贴一个抱抱脸中国对此的[说明推文 - Datasets 使用小贴士: 探索解决数据集无法下载的问题](https://mp.weixin.qq.com/s/e_Krti4U7TxPuFWmUep80Q)。


In [ ]:
datasets= load_dataset("madao33/new-title-chinese")
print(datasets)

#### Loading a Sub-dataset

In [ ]:
# 通过添加第二个参数的方式可以直接加入一个数据集的子任务数据集
boolq_datasets = load_dataset("super_glue","boolq")
print(boolq_datasets)

#### Loading Part of the Dataset

In [ ]:
# 只加载训练集/测试集
trainset1 = load_dataset("madao33/new-title-chinese",split="train")
# 加载指定长度的数据集
trainset2 = load_dataset("madao33/new-title-chinese",split="train[10:100]") 
# 加载指定比例的数据集（这里返回了一个列表）
trainset3 = load_dataset("madao33/new-title-chinese",split=["train[:50%]","validation[10%:]"]) # 注意这里只能是%，并且这里分别表示前50%和后10%

### 5.2 查看数据集

In [ ]:
datasets["train"][:2]

In [ ]:
datasets["train"]["title"][:3]

In [ ]:
datasets["train"].column_names

In [ ]:
datasets["train"].features

### 5.3 数据集的划分（split）, 选取（select）， 过滤（filter）

**划分**：将数据集划分为训练集、测试集等

In [ ]:
# 首先载入数据集并观察一下，发现这个数据集主要有'question'...'label'共四个feature
data = boolq_datasets["train"]
print(data,"\n",data[:2])

In [ ]:
# 可以通过额外添加指令指定数据集划分的比例依据
data.train_test_split(test_size = 0.1, stratify_by_column="label")

**数据集的选取与过滤**

In [ ]:
datasets["train"].select([0,1])

这与上面的直接调取数据查看不同，若用上面的方法`datasets["train"][:3]`直接调用的话则会返回一个python的字典数据；而通过`datasets["train"].select(...)`则仍然会返回一个`Dataset`对象，相当于是抽取了一个子集。

这里数据的筛选还可以通过`filter(lambda example: ...)`函数来进行高级筛选：

In [ ]:
filtered_data = datasets["train"].filter(lambda example: "中国" in example["title"])
print(filtered_data[:2])

### 5.4 数据集的处理（map）

In [ ]:
def add_prefix(example):
    example["title"] = "标题：" + example["title"]
    return example
prefix_data = datasets["train"].map(add_prefix)
print(prefix_data[:2])

可以额外声明:
1. `.map(...,batched = True)`来进行batch数据的处理
2. `.map(...,num_proc = NUM )`来进行多进程处理（NUM为进程数）
3. `.map(...,remove_colomns=[...])`来删除某些列

### 5.5 在线数据集的保存与加载

In [ ]:
processed_datasets.save_to_disk("...")
processed_dataset.load_from_disk("...")

### 5.6 加载本地数据集

In [ ]:
# 直接加载
dataset = load_dataset("csv", data_files = "ADDRESS", split="train")
# 加载csv
dataset = Dataset.from_csv("ADDRESS")
# 加载一个文件夹下的所有csv文件
dataset = load_dataset("csv", data_dir ="ADDRESS", split="train")
# 通过pandas加载（进行格式转换）
import pandas as pd
pd_data = pd.read_csv("ADDRESS")
dataset = Dataset.from_pandas(pd_data)

## Practice 1: 利用AutoClass 拆解 Pipeline

粗浅的说，Pipeline的构成为：
- A Tokenizer instance in charge of mapping raw textual input to token
- A Model instance
- Some (optional) post processing for enhancing model’s output
来源：Pipelines ‒ transformers 3.0.2 documentation

因此我们可以将封装好的这个Pipeline进行拆解，分别自行指定Tokenizer、Model来实现对Pipeline的复现。

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch.nn.functional as F

class MyPipelineForSA(object):
    def __init__(self, checkpoint):
        """
        初始化 MyPipelineForSA 类的实例
        
        参数：
            - checkpoint：预训练模型的路径或名称
        """
        # 预训练模型的分词器 Tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        # 预训练模型（含下游模型）Model
        self.model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    
    def __call__(self, sent_list):
        """
        对给定的句子列表进行情感分析的预测
        
        参数：
            - sent_list：待分析的句子列表
            
        返回：
            - prediction：包含情感分析结果的字典列表
        """
        # 使用预训练模型的分词器对输入文本进行分词和编码，并进行填充
        inputs = self.tokenizer(sent_list, padding=True, return_tensors="pt")
        # 使用预训练模型进行情感分析的前向传播计算
        outputs = self.model(**inputs)
        # 后处理
        # 对输出进行 softmax 操作，得到各类别的概率分布
        outputs = F.softmax(outputs.logits, dim=1)
        # 获取预测结果中每个类别的标签和对应的分数
        id2label = self.model.config.id2label
        prediction = list(map(lambda x: {"label": id2label[x.argmax().item()],
                                          "score": x.max().item()}, outputs))
        return prediction

In [ ]:
# 创建 MyPipelineForSA 类的实例，并指定预训练模型的 checkpoint
classifier = MyPipelineForSA(checkpoint="distilbert-base-uncased-finetuned-sst-2-english")
# 调用实例，对给定的句子列表进行情感分析的预测
result = classifier(["I love HuggingFace!", "I hate it so much!"])
print(result)

## Practice 2: 模型微调

**1. 数据集的加载与清洗**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
# 加载数据集
data = pd.read_csv("ChnSentiCorp_htl_all.csv")
# 数据清洗
data = data.dropna()
data.head()

**2. 创建Dataset**

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    ''' 
    自定义数据集类，继承自 torch.utils.data.Dataset
    '''

    def __init__(self) -> None:
        '''
        初始化 MyDataset 类的实例
        '''
        # 调用父类的初始化方法
        super().__init__() 
        # 加载数据集
        self.data = pd.read_csv("ChnSentiCorp_htl_all.csv")
        # 数据清洗
        self.data = self.data.dropna()

    def __getitem__(self, index):
        '''
        根据给定的索引index, 返回对应的数据
        '''
        # 获取该条数据对应的文本和标签
        text = self.data.iloc[index][0]
        label = self.data.iloc[index][1]
        # 返回该条数据
        return text, label
    
    def __len__(self):
        '''
        返回数据集的总长度
        '''
        return len(self.data)
    
dataset = MyDataset()
for i in range(5):
    print(*dataset[i], sep="\t")

**3. 划分数据集**

In [ ]:
from torch.utils.data import random_split

trainset, validset = random_split(dataset, lengths=[0.1,0.9]) 

**4. 创建DataLoader（数据容器）**

In [ ]:
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
validloader = DataLoader(validset, batch_size=64, shuffle=False)

## 3. Model 模型调用

### 3.1 模型的加载

**直接调用python命令**

In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
# model = AutoModel.from_pretrained("hfl/rbt3", force_download=True)
model = AutoModel.from_pretrained("hfl/rbt3")

Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**加载本地下载好的Model**

In [ ]:
model = AutoModel.from_pretrained("rbt3")

**通过git clone模型仓库**

In [ ]:
!git lfs clone "https://huggingface.co/hfl/rbt3" --include="*.bin"

*通过`lfs ... --include`指定需要下载的文档格式*

### 3.2 模型的config参数

In [ ]:
config = AutoConfig.from_pretrained("hfl/rbt3")
config

BertConfig {
  "_name_or_path": "hfl/rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

### 3.3 模型调用

In [ ]:
sen = "充满鲜花的世界到底在哪里"
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")
inputs = tokenizer(sen, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 1041, 4007, 7831, 5709, 4638,  686, 4518, 1168, 2419, 1762, 1525,
         7027,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
model = AutoModel.from_pretrained("hfl/rbt3", output_attentions=True)

Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
output = model(**inputs)
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1389,  0.2598,  0.7094,  ...,  0.5497, -0.3638, -0.3704],
         [-0.2151, -0.2608,  0.3973,  ..., -0.5889,  0.0549, -0.6675],
         [ 0.4479,  0.6136,  0.4378,  ..., -0.3829,  0.1403, -0.1688],
         ...,
         [-0.2985,  1.0488,  0.4977,  ..., -0.3147,  0.2493,  0.3804],
         [ 0.0230,  0.3447, -0.0087,  ..., -0.2531,  0.2566,  0.2947],
         [ 0.1331,  0.2608,  0.7080,  ...,  0.5577, -0.3658, -0.3668]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-1.4063e-01, -9.9451e-01, -9.9995e-01, -8.4820e-01,  3.9574e-01,
         -5.0347e-01, -1.8221e-01, -3.5995e-01,  7.5781e-01,  9.9639e-01,
          9.6377e-02, -1.0000e+00, -4.0018e-01,  9.9965e-01, -9.9999e-01,
          9.9979e-01,  9.6614e-01,  9.7879e-01, -9.9583e-01,  1.0068e-01,
         -9.9609e-01, -6.1354e-01,  2.4066e-01,  9.9634e-01,  9.7757e-01,
         -9.9047e-01, -9.9921e-01, -1.3862e-01, -9.6030e-01, -9.983

## 5. Dataset

### 5.1 加载在线数据集

In [ ]:
from datasets import *

#### Basic Loading

[HuggingFace_datasets](https://huggingface.co/datasets)

注意，在下载数据集时易出现网络问题，需要科学上网。

这里贴一个抱抱脸中国对此的[说明推文 - Datasets 使用小贴士: 探索解决数据集无法下载的问题](https://mp.weixin.qq.com/s/e_Krti4U7TxPuFWmUep80Q)。


In [ ]:
datasets= load_dataset("madao33/new-title-chinese")
print(datasets)

Found cached dataset csv (C:/Users/xinby/.cache/huggingface/datasets/madao33___csv/madao33--new-title-chinese-5c39318853719c17/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 5850
    })
    validation: Dataset({
        features: ['title', 'content'],
        num_rows: 1679
    })
})


#### Loading a Sub-dataset

In [ ]:
# 通过添加第二个参数的方式可以直接加入一个数据集的子任务数据集
boolq_datasets = load_dataset("super_glue","boolq")
print(boolq_datasets)

Found cached dataset super_glue (C:/Users/xinby/.cache/huggingface/datasets/super_glue/boolq/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3270
    })
    test: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3245
    })
})


#### Loading Part of the Dataset

In [ ]:
# 只加载训练集/测试集
trainset1 = load_dataset("madao33/new-title-chinese",split="train")
# 加载指定长度的数据集
trainset2 = load_dataset("madao33/new-title-chinese",split="train[10:100]") 
# 加载指定比例的数据集（这里返回了一个列表）
trainset3 = load_dataset("madao33/new-title-chinese",split=["train[:50%]","validation[10%:]"]) # 注意这里只能是%，并且这里分别表示前50%和后10%

Found cached dataset csv (C:/Users/xinby/.cache/huggingface/datasets/madao33___csv/madao33--new-title-chinese-5c39318853719c17/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Found cached dataset csv (C:/Users/xinby/.cache/huggingface/datasets/madao33___csv/madao33--new-title-chinese-5c39318853719c17/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Found cached dataset csv (C:/Users/xinby/.cache/huggingface/datasets/madao33___csv/madao33--new-title-chinese-5c39318853719c17/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/2 [00:00<?, ?it/s]

### 5.2 查看数据集

In [ ]:
datasets["train"][:2]

{'title': ['望海楼美国打“台湾牌”是危险的赌博', '大力推进高校治理能力建设'],
 'content': ['近期，美国国会众院通过法案，重申美国对台湾的承诺。对此，中国外交部发言人表示，有关法案严重违反一个中国原则和中美三个联合公报规定，粗暴干涉中国内政，中方对此坚决反对并已向美方提出严正交涉。\n事实上，中国高度关注美国国内打“台湾牌”、挑战一中原则的危险动向。近年来，作为“亲台”势力大本营的美国国会动作不断，先后通过“与台湾交往法”“亚洲再保证倡议法”等一系列“挺台”法案，“2019财年国防授权法案”也多处触及台湾问题。今年3月，美参院亲台议员再抛“台湾保证法”草案。众院议员继而在4月提出众院版的草案并在近期通过。上述法案的核心目标是强化美台关系，并将台作为美“印太战略”的重要伙伴。同时，“亲台”议员还有意制造事端。今年2月，5名共和党参议员致信众议院议长，促其邀请台湾地区领导人在国会上发表讲话。这一动议显然有悖于美国与台湾的非官方关系，其用心是实质性改变美台关系定位。\n上述动向出现并非偶然。在中美建交40周年之际，两国关系摩擦加剧，所谓“中国威胁论”再次沉渣泛起。美国对华认知出现严重偏差，对华政策中负面因素上升，保守人士甚至成立了“当前中国威胁委员会”。在此背景下，美国将台海关系作为战略抓手，通过打“台湾牌”在双边关系中增加筹码。特朗普就任后，国会对总统外交政策的约束力和塑造力加强。其实国会推动通过涉台法案对行政部门不具约束力，美政府在2018年并未提升美台官员互访级别，美军舰也没有“访问”台湾港口，保持着某种克制。但从美总统签署国会通过的法案可以看出，国会对外交产生了影响。立法也为政府对台政策提供更大空间。\n然而，美国需要认真衡量打“台湾牌”成本。首先是美国应对危机的代价。美方官员和学者已明确发出警告，美国卷入台湾问题得不偿失。美国学者曾在媒体发文指出，如果台海爆发危机，美国可能需要“援助”台湾，进而导致新的冷战乃至与中国大陆的冲突。但如果美国让台湾自己面对，则有损美国的信誉，影响美盟友对同盟关系的支持。其次是对中美关系的危害。历史证明，中美合则两利、斗则两伤。中美关系是当今世界最重要的双边关系之一，保持中美关系的稳定发展，不仅符合两国和两国人民的根本利益，也是国际社会的普遍期待。美国蓄意挑战台湾问题的底线，加剧中美关系的复杂性和不确定

In [ ]:
datasets["train"]["title"][:3]

['望海楼美国打“台湾牌”是危险的赌博', '大力推进高校治理能力建设', '坚持事业为上选贤任能']

In [ ]:
datasets["train"].column_names

['title', 'content']

In [ ]:
datasets["train"].features

{'title': Value(dtype='string', id=None),
 'content': Value(dtype='string', id=None)}

### 5.3 数据集的划分（split）, 选取（select）， 过滤（filter）

**划分**：将数据集划分为训练集、测试集等

In [ ]:
# 首先载入数据集并观察一下，发现这个数据集主要有'question'...'label'共四个feature
data = boolq_datasets["train"]
print(data,"\n",data[:2])

Dataset({
    features: ['question', 'passage', 'idx', 'label'],
    num_rows: 9427
}) 
 {'question': ['do iran and afghanistan speak the same language', 'do good samaritan laws protect those who help at an accident'], 'passage': ['Persian language -- Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.', "Good Samaritan law -- Good Samaritan laws offer legal protection to people who give reasonable assistance to those who are, or who they believe to be, injured

In [ ]:
# 可以通过额外添加指令指定数据集划分的比例依据
data.train_test_split(test_size = 0.1, stratify_by_column="label")

DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 8484
    })
    test: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 943
    })
})

**数据集的选取与过滤**

In [ ]:
datasets["train"].select([0,1])

Dataset({
    features: ['title', 'content'],
    num_rows: 2
})

这与上面的直接调取数据查看不同，若用上面的方法`datasets["train"][:3]`直接调用的话则会返回一个python的字典数据；而通过`datasets["train"].select(...)`则仍然会返回一个`Dataset`对象，相当于是抽取了一个子集。

这里数据的筛选还可以通过`filter(lambda example: ...)`函数来进行高级筛选：

In [ ]:
filtered_data = datasets["train"].filter(lambda example: "中国" in example["title"])
print(filtered_data[:2])

Loading cached processed dataset at C:\Users\xinby\.cache\huggingface\datasets\madao33___csv\madao33--new-title-chinese-5c39318853719c17\0.0.0\eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d\cache-2d4b892f9bd3b103.arrow


{'title': ['聚焦两会，世界探寻中国成功秘诀', '望海楼中国经济的信心来自哪里'], 'content': ['世界眼中的中国两会，是一道集聚全国人民磅礴之力的风景线，不仅展现着新时代中国的意气风发，而且增添着世界的生机和动能\n\u3000\u3000两会时间，世界研究中国的重要时间。\n中国智慧、中国方略、中国成就集中展现，来自世界的热评此起彼伏。“中国两会对全球事务有着巨大影响力”“是时候对中国的突飞猛进有清醒的认识了”“中国方案为世界提供借鉴”……来自世界的这些叙述，折射着探寻成功秘诀的目光。各国媒体纷纷向北京增派记者，3000多名中外记者报名采访中国两会。世界瞩目中国两会，期待进一步读懂新时代的中国。\n作为世界最大发展中国家和第二大经济体，中国不断创造着人类发展史上惊天动地的奇迹。中国提供着最高的对世界经济增长的贡献率、最高的对世界减贫事业的贡献率，中国建成了世界上最大的社会保障网、高速铁路网，中国科技创新在诸多领域实现并跑、领跑……国际舆论普遍认为，着眼自身治理能力现代化的中国，日益为全球治理作出重要贡献。\n世界经济复苏发展，中国功不可没。中国国家统计局最新数据显示，2017年，中国国内生产总值增速达6.9%，占世界经济的比重达15%，稳居世界第二。“中国的成功故事与世界经济的命运紧密相连。”国际货币基金组织第一副总裁戴维·利普顿这样评价。“世行对中国经济发展有信心，对中国持续成为全球经济增长引擎有信心。”世界银行发展预测局局长阿伊汗·高斯为中国点赞。\n国外不少人感叹，世界太需要像中国这样不断书写成功故事。但是，环顾当下的世界，一些国家深陷矛盾丛生、乱象频发的怪圈。在今年年初的达沃斯论坛年会和慕尼黑安全会议上，来自西方一些国家的人士纷纷大谈对社会分裂、政治极化、民粹主义盛行之忧。国际政治、社会领域诸多乱象，从若干侧面说明了失却方向之痛、道路探索之难。与此形成鲜明反差的是，中国的稳步发展和繁荣景象，无疑给不乏深忧的世界注入了昂扬振奋的力量。\n中国的成功故事，揭示了中国道路自信、理论自信、制度自信、文化自信之源。中共十九大的胜利召开，为中国建设现代化强国指明了方向，为中华民族伟大复兴做了全面谋划。世界瞩目，十九大后的首次中国两会将如何围绕民众普遍关心的反腐倡廉、社会保障、教育公平、医疗改革、脱贫攻坚、依法治国、改革开放等一系列议题谋篇布局

### 5.4 数据集的处理（map）

In [ ]:
def add_prefix(example):
    example["title"] = "标题：" + example["title"]
    return example
prefix_data = datasets["train"].map(add_prefix)
print(prefix_data[:2])

Map:   0%|          | 0/5850 [00:00<?, ? examples/s]

{'title': ['标题：望海楼美国打“台湾牌”是危险的赌博', '标题：大力推进高校治理能力建设'], 'content': ['近期，美国国会众院通过法案，重申美国对台湾的承诺。对此，中国外交部发言人表示，有关法案严重违反一个中国原则和中美三个联合公报规定，粗暴干涉中国内政，中方对此坚决反对并已向美方提出严正交涉。\n事实上，中国高度关注美国国内打“台湾牌”、挑战一中原则的危险动向。近年来，作为“亲台”势力大本营的美国国会动作不断，先后通过“与台湾交往法”“亚洲再保证倡议法”等一系列“挺台”法案，“2019财年国防授权法案”也多处触及台湾问题。今年3月，美参院亲台议员再抛“台湾保证法”草案。众院议员继而在4月提出众院版的草案并在近期通过。上述法案的核心目标是强化美台关系，并将台作为美“印太战略”的重要伙伴。同时，“亲台”议员还有意制造事端。今年2月，5名共和党参议员致信众议院议长，促其邀请台湾地区领导人在国会上发表讲话。这一动议显然有悖于美国与台湾的非官方关系，其用心是实质性改变美台关系定位。\n上述动向出现并非偶然。在中美建交40周年之际，两国关系摩擦加剧，所谓“中国威胁论”再次沉渣泛起。美国对华认知出现严重偏差，对华政策中负面因素上升，保守人士甚至成立了“当前中国威胁委员会”。在此背景下，美国将台海关系作为战略抓手，通过打“台湾牌”在双边关系中增加筹码。特朗普就任后，国会对总统外交政策的约束力和塑造力加强。其实国会推动通过涉台法案对行政部门不具约束力，美政府在2018年并未提升美台官员互访级别，美军舰也没有“访问”台湾港口，保持着某种克制。但从美总统签署国会通过的法案可以看出，国会对外交产生了影响。立法也为政府对台政策提供更大空间。\n然而，美国需要认真衡量打“台湾牌”成本。首先是美国应对危机的代价。美方官员和学者已明确发出警告，美国卷入台湾问题得不偿失。美国学者曾在媒体发文指出，如果台海爆发危机，美国可能需要“援助”台湾，进而导致新的冷战乃至与中国大陆的冲突。但如果美国让台湾自己面对，则有损美国的信誉，影响美盟友对同盟关系的支持。其次是对中美关系的危害。历史证明，中美合则两利、斗则两伤。中美关系是当今世界最重要的双边关系之一，保持中美关系的稳定发展，不仅符合两国和两国人民的根本利益，也是国际社会的普遍期待。美国蓄意挑战台湾问题的底线，加剧中美关系的复杂

可以额外声明:
1. `.map(...,batched = True)`来进行batch数据的处理
2. `.map(...,num_proc = NUM )`来进行多进程处理（NUM为进程数）
3. `.map(...,remove_colomns=[...])`来删除某些列

### 5.5 在线数据集的保存与加载

In [ ]:
processed_datasets.save_to_disk("...")
processed_dataset.load_from_disk("...")

### 5.6 加载本地数据集

In [ ]:
# 直接加载
dataset = load_dataset("csv", data_files = "ADDRESS", split="train")
# 加载csv
dataset = Dataset.from_csv("ADDRESS")
# 加载一个文件夹下的所有csv文件
dataset = load_dataset("csv", data_dir ="ADDRESS", split="train")
# 通过pandas加载（进行格式转换）
import pandas as pd
pd_data = pd.read_csv("ADDRESS")
dataset = Dataset.from_pandas(pd_data)

## Practice 1: 利用AutoClass 拆解 Pipeline

粗浅的说，Pipeline的构成为：
- A Tokenizer instance in charge of mapping raw textual input to token
- A Model instance
- Some (optional) post processing for enhancing model’s output
来源：Pipelines ‒ transformers 3.0.2 documentation

因此我们可以将封装好的这个Pipeline进行拆解，分别自行指定Tokenizer、Model来实现对Pipeline的复现。

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch.nn.functional as F

class MyPipelineForSA(object):
    def __init__(self, checkpoint):
        """
        初始化 MyPipelineForSA 类的实例
        
        参数：
            - checkpoint：预训练模型的路径或名称
        """
        # 预训练模型的分词器 Tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        # 预训练模型（含下游模型）Model
        self.model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    
    def __call__(self, sent_list):
        """
        对给定的句子列表进行情感分析的预测
        
        参数：
            - sent_list：待分析的句子列表
            
        返回：
            - prediction：包含情感分析结果的字典列表
        """
        # 使用预训练模型的分词器对输入文本进行分词和编码，并进行填充
        inputs = self.tokenizer(sent_list, padding=True, return_tensors="pt")
        # 使用预训练模型进行情感分析的前向传播计算
        outputs = self.model(**inputs)
        # 后处理
        # 对输出进行 softmax 操作，得到各类别的概率分布
        outputs = F.softmax(outputs.logits, dim=1)
        # 获取预测结果中每个类别的标签和对应的分数
        id2label = self.model.config.id2label
        prediction = list(map(lambda x: {"label": id2label[x.argmax().item()],
                                          "score": x.max().item()}, outputs))
        return prediction

In [ ]:
# 创建 MyPipelineForSA 类的实例，并指定预训练模型的 checkpoint
classifier = MyPipelineForSA(checkpoint="distilbert-base-uncased-finetuned-sst-2-english")
# 调用实例，对给定的句子列表进行情感分析的预测
result = classifier(["I love HuggingFace!", "I hate it so much!"])
print(result)

[{'label': 'POSITIVE', 'score': 0.9998130202293396}, {'label': 'NEGATIVE', 'score': 0.9995473027229309}]


## Practice 2: 模型微调

**1. 数据集的加载与清洗**

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
# 加载数据集
data = pd.read_csv("ChnSentiCorp_htl_all.csv")
# 数据清洗
data = data.dropna()
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


**2. 创建Dataset**

In [14]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    ''' 
    自定义数据集类，继承自 torch.utils.data.Dataset
    '''

    def __init__(self) -> None:
        '''
        初始化 MyDataset 类的实例
        '''
        # 调用父类的初始化方法
        super().__init__() 
        # 加载数据集
        self.data = pd.read_csv("ChnSentiCorp_htl_all.csv")
        # 数据清洗
        self.data = self.data.dropna()

    def __getitem__(self, index):
        '''
        根据给定的索引index, 返回对应的数据
        '''
        # 获取该条数据对应的文本和标签
        text = self.data.iloc[index][0]
        label = self.data.iloc[index][1]
        # 返回该条数据
        return text, label
    
    def __len__(self):
        '''
        返回数据集的总长度
        '''
        return len(self.data)
    
dataset = MyDataset()
for i in range(5):
    print(*dataset[i], sep="\t")

1	距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.
1	商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
1	早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
1	宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~
1	CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风


**3. 划分数据集**

In [16]:
from torch.utils.data import random_split

trainset, validset = random_split(dataset, lengths=[0.1,0.9]) 

ValueError: Sum of input lengths does not equal the length of the input dataset!

**4. 创建DataLoader（数据容器）**

In [ ]:
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
validloader = DataLoader(validset, batch_size=64, shuffle=False)

/Users/xinby/opt/anaconda3/envs/transformer_mac/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'trainset' is not defined